In [90]:
import pandas as pd
from z3 import *
from time import time
from copy import copy

# Define floors
floors = set(range(4, 22))
male_floors = set([8, 10, 16, 19])
female_floors = set([7, 14, 18, 20])
mixed_floors = floors.difference(male_floors).difference(female_floors)
rf_floors = set(range(5, 22, 4))
laundry_floors = set([9, 17])
freshmen_floors = set([6, 8, 11, 14, 16, 17, 20])
senior_floors = floors.difference(freshmen_floors)

# Define room positions
corridor_nonaircon_pos = set(map(str, range(101, 107) + range(115, 120)))
corridor_aircon_pos = set(map(str, range(109, 115)))
suite_room_letters = map(lambda x: chr(ord('A') + x), range(6))
suite_nonaircon_pos = set([str(suite_num) + letter for suite_num in [100, 107] for letter in suite_room_letters])
suite_aircon_pos = set([str(suite_num) + letter for suite_num in [108] for letter in suite_room_letters])
aircon_pos = corridor_aircon_pos.union(suite_aircon_pos)
nonaircon_pos = corridor_nonaircon_pos.union(suite_nonaircon_pos)
corridor_pos = corridor_nonaircon_pos.union(corridor_aircon_pos)
suite_room_pos = suite_nonaircon_pos.union(suite_aircon_pos)
absent_laundry_pos = set(map(str, range(103, 107)))
room_pos = corridor_pos.union(suite_room_pos)


# Rooms and floor query methods

def get_valid_floors(include_male_floors=True, include_female_floors=True, include_mixed_floors=True, include_freshman_floors=True, include_senior_floors=True):
    gender_floors = male_floors if include_male_floors else set()
    gender_floors = gender_floors.union(female_floors) if include_female_floors else gender_floors
    gender_floors = gender_floors.union(mixed_floors) if include_mixed_floors else gender_floors

    seniority_floors = freshmen_floors if include_freshman_floors else set()
    seniority_floors = seniority_floors.union(senior_floors) if include_senior_floors else seniority_floors

    return gender_floors.intersection(seniority_floors)


def get_valid_pos(floor_num, include_suites=True, include_corridors=True, include_aircon=True, include_non_aircon=True):
    positions = set()
    positions = positions.union(suite_room_pos) if include_suites else positions
    positions = positions.union(corridor_pos) if include_corridors else positions
    positions = positions.difference(aircon_pos) if not include_aircon else positions
    positions = positions.difference(nonaircon_pos) if not include_non_aircon else positions
        
    if floor_num not in floors:
        positions = positions.difference(room_pos)
        
    if floor_num in rf_floors:
        positions = positions.difference(suite_aircon_pos)
    
    if floor_num in laundry_floors:
        positions = positions.difference(absent_laundry_pos)

    return positions


# Returns a set of preferred floors for a person
def get_preferred_floors(is_male=True, is_freshmen=False, include_single_gen_floors=True, include_mixed_floors=True):
    include_male_floors = is_male and include_single_gen_floors
    include_female_floors = !is_male and include_single_gen_floors
    include_freshman_floors = is_freshmen
    include_senior_floors = !is_freshmen
    
    return get_valid_floors(include_male_floors, include_female_floors, include_mixed_floors, include_freshman_floors, include_senior_floors)
    
    

def get_valid_rooms(preferred_floors, preferred_positions):
    return set([str(floor_num) + '-' + pos for floor_num in preferred_floors for pos in preferred_positions])  



def get_all_rooms():
    rooms = []
    for floor_num in floors:
        if floor_num in laundry_floors:
            rooms.extend([str(floor_num) + '-' + pos for pos in room_pos.difference(absent_laundry_pos)])
        elif floor_num in rf_floors:
            rooms.extend([str(floor_num) + '-' + pos for pos in room_pos.difference(suite_aircon_pos)])
        else:
            rooms.extend([str(floor_num) + '-' + pos for pos in room_pos])
    return rooms
    


In [116]:
class Person:
    def __init__(self, entry_id, gender, person_type, room_pref1, room_pref2, room_pref3, preference_comment, pref_floor_gender):
        self.entry_id = entry_id
        self.gender = gender
        self.is_male = self.gender == "Male"
        self.person_type = person_type
        self.is_senior = person_type == "USP Senior UG"
        self.room_pref1 = RoomType.make_room_type(room_pref1)
        self.room_pref2 = RoomType.make_room_type(room_pref2)
        self.room_pref3 = RoomType.make_room_type(room_pref3)
        self.preference_comment = preference_comment
        self.pref_floor_gender = pref_floor_gender
        
    # Need to abstract this away into a RoomType or Preference class   
    def valid_rooms(self):
        return get_valid_rooms(self.valid_floors(), self.valid_pos())
    
    def valid_pos(self):
        return get_valid_pos(16, self.room_pref1.is_suite, not self.room_pref1.is_suite, self.room_pref1.is_aircon, not self.room_pref1.is_aircon)

    # Only one set of valid floors for one person
    def valid_floors(self):
        include_single_gen_floors = self.pref_floor_gender == "Single Gender Floor" or self.pref_floor_gender == "No Preference"
        include_mixed_floors = self.pref_floor_gender == "Mixed Gender Floor" or self.pref_floor_gender == "No Preference"
        return get_preferred_floors(self.is_male, not self.is_senior, include_single_gen_floors, include_mixed_floors)
                                    
                                    

In [117]:
class Room:
    def __init__(self, level, position):
        self.level = level
        self.position = position
        self.room_type = RoomType(self.is_suite(), self.is_aircon())
        
    def __init__(self, room_string):
        self.level = room_string.split('-')[0]
        self.position = room_string.split('-')[1]
        self.room_type = RoomType(self.is_suite(), self.is_aircon())

        
    def __str__(self):
        return str(self.level) + '-' + str(self.position)

    def is_aircon(self):
        return self.position in aircon_pos
    
    def is_suite(self):
        return self.position in suite_room_pos
    
    def get_type(self):
        return self.room_type.get_attributes()

    def get_inverse_type(self):
        return self.room_type.get_inverse_attributes()


class RoomType:
    def __init__(self, is_suite, is_aircon):
        self.is_suite = is_suite
        self.is_aircon = is_aircon
        
    # Initialises from description Eg. USP, Single (6 bdrm Apt, Non Air-Con)
    @staticmethod 
    def make_room_type(description):
        is_suite = '6 bdrm Apt' in description
        is_aircon = 'Non Air-Con' not in description
        return RoomType(is_suite, is_aircon)
        

    def __str__(self):
        return 'Suite: {}, Aircon: {} '.format(self.is_suite, self.is_aircon)

In [118]:
def solve(nrows=None):
    input_data_frame = pd.read_csv('data/usprc.csv', nrows=nrows, dtype=object).fillna('').values
    constraints = []
    standard_implications = []
    preferences = []
    opt = Optimize()

    # Populate people
    all_people = []
    for row in input_data_frame:
        person = Person(*row[:8])
        all_people.append(person)
       
    # Populate rooms
    all_rooms = map(Room, get_all_rooms())
        
    
    # Populate all room possible assignments
    assignments = {}
    for person in all_people:
        assignments[person] = {}

        for room in all_rooms:
            possibility = Bool("{} gets {}".format(person.entry_id, room))
            
            # Add possibility to possible assignments
            assignments[person][room] = possibility
            
    # One person can only get one room
    for person in all_people:
        constraints.append(Or(assignments[person].values()))

    # One room can only be assigned to one person
    for person in all_people:
        for room in all_rooms:
            for other_person in all_people:
                if other_person == person:
                    continue
            
                implication = Implies(assignments[person][room], Not(assignments[other_person][room]))
                constraints.append(implication)
    
    
    # Every person gets his/her preferred floor type (gender)
    for person in all_people:
        possible_assignments = []
        for room in person.valid_rooms():
            possible_assignments.append(Bool("{} gets {}".format(person.entry_id, room)))
        preferences.append(Or(possible_assignments))
    
    
    # Solve
    for constraint in constraints:
        opt.add(constraint)
    
    for implication in standard_implications:
        opt.add(implication)
    
    for preference in preferences:
        opt.add(preference)
        
        
    opt.check()
    model = opt.model()
    
    all_symbols = [assignments[person][room] for person in all_people for room in all_rooms]
    return filter(lambda x: is_true(model.eval(x)), all_symbols)

In [119]:
%time solve(10)

CPU times: user 7 s, sys: 151 ms, total: 7.15 s
Wall time: 7.62 s


[132083 gets 12-109,
 133621 gets 7-118,
 131858 gets 18-111,
 105327 gets 19-112,
 167885 gets 7-100F,
 167320 gets 18-107F,
 168255 gets 12-107D,
 171415 gets 18-100D,
 170839 gets 15-100D,
 101335 gets 12-114]

In [174]:
pd.read_csv('data/usprc.csv', nrows=10, dtype=object).fillna('')

,Entry ID,Gender Description,Classification Description 9,Room Preference Description 1,Room Preference Description 2,Room Preference Description 3,Room Preference Comments,Floor Gender Preference,Nationality Description 4,PR Status,Admit Term,Enrollment Term,Faculty,Medical,Enrollment Status
0,132083,Male,USP Senior UG,"USP, Single (Corridor, Air-Con)","USP, Single (6 bdrm Apt, Air-Con)","USP, Single (6 bdrm Apt, Non Air-Con)",15100,Mixed Gender Floor,SINGAPORE CITIZEN,FALSE,1510,4,ARTS & SOCIAL SCIENCES,,CURRENT
1,133621,Female,USP Senior UG,"USP, Single (Corridor, Non Air-Con)","USP, Single (6 bdrm Apt, Non Air-Con)",,#18-103 or #07-103. facing CAPT.,Single Gender Floor,SINGAPORE CITIZEN,FALSE,1510,4,SCIENCE,,CURRENT
2,131858,Female,USP Senior UG,"USP, Single (Corridor, Air-Con)","USP, Single (6 bdrm Apt, Air-Con)",,#18-110,Single Gender Floor,SINGAPORE CITIZEN,FALSE,1510,4,ARTS & SOCIAL SCIENCES,Respiratory/breathing difficulties if the air ...,CURRENT
3,105327,Male,USP Senior UG,"USP, Single (Corridor, Air-Con)",,,#19-113 my current room,Single Gender Floor,Indonesian,FALSE,1410,6,SCIENCE,,CURRENT
4,167885,Female,USP Senior UG,"USP, Single (6 bdrm Apt, Non Air-Con)","USP, Single (6 bdrm Apt, Air-Con)","USP, Single (Corridor, Air-Con)",#feefivefuudfam,No Preference,SINGAPORE CITIZEN,FALSE,1610,2,ARTS & SOCIAL SCIENCES,,CURRENT
5,167320,Female,USP Senior UG,"USP, Single (6 bdrm Apt, Non Air-Con)","USP, Single (6 bdrm Apt, Air-Con)","USP, Single (Corridor, Non Air-Con)",#feefivefuudfam,No Preference,SINGAPORE CITIZEN,FALSE,1610,2,ARTS & SOCIAL SCIENCES,,CURRENT
6,168255,Female,USP Senior UG,"USP, Single (6 bdrm Apt, Non Air-Con)","USP, Single (6 bdrm Apt, Air-Con)","USP, Single (Corridor, Non Air-Con)",#feefivefuudfam,No Preference,SINGAPORE CITIZEN,FALSE,1610,2,ARTS & SOCIAL SCIENCES,,CURRENT
7,171415,Female,USP Senior UG,"USP, Single (6 bdrm Apt, Non Air-Con)","USP, Single (6 bdrm Apt, Air-Con)","USP, Single (Corridor, Non Air-Con)",#feefivefuudfam,No Preference,SINGAPORE CITIZEN,FALSE,1610,2,ARTS & SOCIAL SCIENCES,,CURRENT
8,170839,Female,USP Senior UG,"USP, Single (6 bdrm Apt, Non Air-Con)","USP, Single (Corridor, Non Air-Con)",,#feefivefuudfam,No Preference,SINGAPORE CITIZEN,FALSE,1610,2,ARTS & SOCIAL SCIENCES,,CURRENT
9,101335,Male,USP Senior UG,"USP, Single (Corridor, Air-Con)","USP, Single (6 bdrm Apt, Air-Con)",,#greenhousebesthouse OR #seniorfloor15,Mixed Gender Floor,SINGAPORE CITIZEN,FALSE,1410,6,ENGINEERING,Nil,CURRENT


1

hi 1
